In [1]:
import numpy as np
from numpy import linalg as la
import time
import re
import dgl
from pandas import DataFrame
import torch
from torch.nn import MSELoss
from torch.nn import functional as F
from IPython.display import display

import src.dag_utils as dagu
from src.arch import DAGConv, FB_DAGConv
from src.models import Model, AlternatingModel
from src.baselines_archs import GCNN_2L, GAT, MLP

# Ser random seed
SEED = 10
np.random.seed(SEED)
torch.manual_seed(SEED)
dgl.random.seed(SEED)

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

### Default parameters and auxiliary function

In [2]:
# ### Best Params for DAGConv
# Data parameters
M = 2000
data_p = {
    'n_tries': 15,

    ## Graph parameters
    'p': 0.2,                    # Edge prob in Erdos-Renyi DAG
    'N': 100,                    # Number of nodes

    ## Signal parameters
    'M': M,                   # Number of observed signals
    'M_train': int(0.7 * M),  # Samples selected for training
    'M_val': int(0.2 * M),    # Samples selected for validation
    'M_test': int(0.1 * M),   # Samples selected for test
    'max_src_node': 5,           # Maximum index of nodes allowed to be sources
    'n_sources': 3,              # Maximum Number of source nodes
    'n_p': 0.05,                 # Normalized noise power
    'max_GSO': 100,              # Maximum index of GSOs involved in the diffusion
    'min_GSO': 50,               # Minimum index of GSOs involved in the diffusion
    'n_GSOs': 25                 # Number of GSOs
}

In [3]:
# Model parameters
default_arch_p = {
    'arch': DAGConv,    # Selected architecture
    'in_dim': 1,        # Input dimension
    'hid_dim': 128,  # 64 works too     # Hidden dimension
    'out_dim': 1,       # Output dimension
    'L': 2,             # Number of layers
    'bias': True
}

default_mod_p = {
    'loss': MSELoss(),  # Loss function
    'bs': 25,           # Size of the batch
    'lr': 5e-4,         # Learning rate
    'epochs': 35,       # Number of training epochs 
    'pat': 15,          # Number of non-decreasing epoch to stop training
    'wd': 1e-4,         # Weight decay
    'alt': False,       # Train weights in alternating fashion
    'e_h': 1,           # Number of iters for learning h
    'e_W': 1,           # Number of iter for learning W
}

In [4]:
# ### Best Params for FB_DAGConv
# # Model parameters
# default_arch_p = {
#     'arch': FB_DAGConv,    # Selected architecture
#     'in_dim': 1,        # Input dimension
#     'hid_dim': 64,     # Hidden dimension
#     'out_dim': 1,       # Output dimension
#     'L': 2,  # 3 also works well          # Number of layers
#     'bias': True
# }

# default_mod_p = {
#     'loss': MSELoss(),  # Loss function
#     'bs': 25,           # Size of the batch
#     'lr': 5e-4,         # Learning rate
#     'epochs': 50,       # Number of training epochs 
#     'pat': 15,          # Number of non-decreasing epoch to stop training
#     'wd': 1e-5,         # Weight decay
#     'alt': False,       # Train weights in alternating fashion
#     'e_h': 1,           # Number of iters for learning h
#     'e_W': 1,           # Number of iter for learning W
# }

In [5]:
def run_exp(d_p, d_arc_p, d_mod_p, exps):
    err = np.zeros((d_p['n_tries'], len(exps)))
    std = np.zeros((d_p['n_tries'], len(exps)))
    times = np.zeros((d_p['n_tries'], len(exps)))
    t_begin = time.time()
    for i in range(d_p['n_tries']):
        # Create adj, transitive closure and GSOs for the DAG
        Adj, dag = dagu.create_dag(d_p['N'], d_p['p'])
        W = la.inv(np.eye(d_p['N']) - Adj)
        W_inf = la.inv(W)
        GSOs = np.array([(W * dagu.compute_Dq(dag, i, d_p['N'])) @ W_inf for i in range(d_p['N'])])

        # Create signals
        range_GSO = np.arange(d_p['min_GSO'], d_p['max_GSO'])
        gsos_idx = np.random.choice(range_GSO, size=d_p['n_GSOs'], replace=False)
        sel_GSOs = GSOs[gsos_idx]
        Y_t, X_t = dagu.create_diff_data(d_p['M'], sel_GSOs, d_p['max_src_node'], d_p['n_p'],
                                         d_p['n_sources'], torch_tensor=True)

        X_data = {'train': X_t[:d_p['M_train']], 'val': X_t[d_p['M_train']:-d_p['M_test']], 'test': X_t[-d_p['M_test']:]}
        Y_data = {'train': Y_t[:d_p['M_train']], 'val': Y_t[d_p['M_train']:-d_p['M_test']], 'test': Y_t[-d_p['M_test']:]}
    
        for j, exp in enumerate(exps):
            arc_p = {**d_arc_p, **exp['arc_p']}
            mod_p = {**d_mod_p, **exp['mod_p']}

            used_GSOs = torch.Tensor(sel_GSOs) if exp['sel_GSOs'] else torch.Tensor(GSOs)
            arch = arc_p['arch'](arc_p['in_dim'], arc_p['hid_dim'], arc_p['out_dim'], arc_p['L'], used_GSOs,
                                 arc_p['bias'])

            if mod_p['alt']:
                model = AlternatingModel(arch, mod_p['loss'], mod_p['e_h'], mod_p['e_W'],
                                         device=device)
            else:
                model = Model(arch, mod_p['loss'], device=device)

            t_i = time.time()
            model.fit(X_data, Y_data, mod_p['lr'], mod_p['epochs'], mod_p['bs'],
                      mod_p['wd'], patience=mod_p['pat'])
            t_e = time.time() - t_i
            err[i,j], std[i,j] = model.test(X_data['test'], Y_data['test'])
            times[i,j] = t_e

            print(f'- {i}, {j}. err: {err[i,j]:.3f} - std: {std[i,j]:.3f} - time: {times[i,j]:.1f}')

    total_t = (time.time() - t_begin)/60
    print(f'----- Ellapsed time: {total_t:.2f} minutes -----')
    return err, std, times

def display_data(exps_leg, err, std, time):
    data = {
        'Exp': exps_leg,
        'Mean Err': err.mean(axis=0),
        'Median Err': np.median(err, axis=0),
        'Mean Std': std.mean(axis=0),
        'time': time.mean(axis=0)
    }
    df = DataFrame(data)
    display(df)

def read_output(exps, n_tries):
    err = np.zeros((n_tries, len(exps)))
    std = np.zeros((n_tries, len(exps)))
    times = np.zeros((n_tries, len(exps)))

    with open('output.txt', 'r') as file:
        for line in file:
            match = re.match(r'- (\d+), (\d+)\. err: ([\d.]+) - std: ([\d.]+) - time: ([\d.]+)', line)

            if match:
                i = int(match.group(1))
                j = int(match.group(2))
                err_val = float(match.group(3))
                std_val = float(match.group(4))
                time_val = float(match.group(5))

                err[i,j] = err_val
                std[i,j] = std_val
                times[i,j] = time_val

    return err, std, times

### Experiments

#### Learning Rate

In [6]:
#####   Learning Rate   #####
Exps = [
    {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'lr': 1e-4}},
    {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'lr': 5e-4}},
    {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'lr': 1e-3}},
    {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'lr': 5e-3}},
    {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'lr': 1e-2}},

    {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'lr': 1e-4}},
    {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'lr': 5e-4}},
    {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'lr': 1e-3}},
    {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'lr': 5e-3}},
    {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'lr': 1e-2}},
]

err, std, times = run_exp(data_p, default_arch_p, default_mod_p, Exps)
leg = [f'{exp["sel_GSOs"]}-{exp["mod_p"]["lr"]}'  for exp in Exps]
display_data(leg, err, std, times)

epoch 0
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100 1
2500
100 25 100 128
320000
100 25 100

Exception: JA

#### Weight Decay

In [7]:
#####   Weight Decay   #####
Exps = [
    {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'wd': 0}},
    {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'wd': 1e-7}},
    {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'wd': 1e-6}},
    {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'wd': 1e-5}},
    {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'wd': 1e-4}},
    {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'wd': 1e-2}},

    {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'wd': 0}},
    {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'wd': 1e-7}},
    {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'wd': 1e-6}},
    {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'wd': 1e-5}},
    {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'wd': 1e-4}},
    {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'wd': 1e-2}},
]

err, std, times = run_exp(data_p, default_arch_p, default_mod_p, Exps)
leg = [f'{exp["sel_GSOs"]}-{exp["mod_p"]["wd"]}'  for exp in Exps]
display_data(leg, err, std, times)

- 0, 0. err: 0.035 - std: 0.034 - time: 7.0
- 0, 1. err: 0.029 - std: 0.034 - time: 7.0
- 0, 2. err: 0.029 - std: 0.038 - time: 7.0
- 0, 3. err: 0.032 - std: 0.035 - time: 7.0
- 0, 4. err: 0.033 - std: 0.033 - time: 6.4
- 0, 5. err: 0.094 - std: 0.043 - time: 7.0
- 0, 6. err: 0.030 - std: 0.039 - time: 2.4
- 0, 7. err: 0.031 - std: 0.039 - time: 2.4
- 0, 8. err: 0.034 - std: 0.041 - time: 1.8
- 0, 9. err: 0.035 - std: 0.037 - time: 2.4
- 0, 10. err: 0.038 - std: 0.037 - time: 2.4
- 0, 11. err: 0.145 - std: 0.038 - time: 1.1
- 1, 0. err: 0.019 - std: 0.023 - time: 7.0
- 1, 1. err: 0.019 - std: 0.027 - time: 7.0
- 1, 2. err: 0.018 - std: 0.025 - time: 7.0
- 1, 3. err: 0.019 - std: 0.025 - time: 7.0
- 1, 4. err: 0.021 - std: 0.025 - time: 7.0
- 1, 5. err: 0.114 - std: 0.045 - time: 7.0
- 1, 6. err: 0.019 - std: 0.026 - time: 2.4
- 1, 7. err: 0.018 - std: 0.025 - time: 2.4
- 1, 8. err: 0.019 - std: 0.026 - time: 3.0
- 1, 9. err: 0.020 - std: 0.027 - time: 2.8
- 1, 10. err: 0.020 - std: 0.0

,Exp,Mean Err,Median Err,Mean Std,time
0,False-0,0.035369,0.026251,0.025919,6.498950
1,False-1e-07,0.035642,0.027780,0.025711,6.869843
2,False-1e-06,0.034860,0.026935,0.026355,6.402999
3,False-1e-05,0.035240,0.027124,0.026675,6.847422
4,False-0.0001,0.036718,0.027734,0.026762,6.577748
5,False-0.01,0.075936,0.055504,0.033147,4.683950
6,True-0,0.035855,0.028168,0.027166,2.532184
7,True-1e-07,0.035064,0.027000,0.026876,2.433960
8,True-1e-06,0.035966,0.031504,0.027409,2.374039
9,True-1e-05,0.036171,0.028270,0.027638,2.407484


####  Number of Epochs

In [8]:
#####   Number of Epochs   #####
Exps = [
    {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'epochs': 15}},
    {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'epochs': 20}},
    {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'epochs': 35}},
    {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'epochs': 50}},
    {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'epochs': 75}},
    {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'epochs': 100}},
    {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'epochs': 200}},

    {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'epochs': 15}},
    {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'epochs': 20}},
    {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'epochs': 35}},
    {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'epochs': 50}},
    {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'epochs': 75}},
    {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'epochs': 100}},
    {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'epochs': 200}},
]

err, std, times = run_exp(data_p, default_arch_p, default_mod_p, Exps)
leg = [f'{exp["sel_GSOs"]}-{exp["mod_p"]["epochs"]}'  for exp in Exps]
display_data(leg, err, std, times)

- 0, 0. err: 0.045 - std: 0.026 - time: 3.0
- 0, 1. err: 0.045 - std: 0.026 - time: 4.0
- 0, 2. err: 0.044 - std: 0.026 - time: 7.0
- 0, 3. err: 0.045 - std: 0.025 - time: 5.4
- 0, 4. err: 0.047 - std: 0.027 - time: 6.4
- 0, 5. err: 0.045 - std: 0.026 - time: 5.4
- 0, 6. err: 0.044 - std: 0.026 - time: 8.0
- 0, 7. err: 0.045 - std: 0.026 - time: 1.3
- 0, 8. err: 0.045 - std: 0.026 - time: 1.5
- 0, 9. err: 0.045 - std: 0.026 - time: 2.9
- 0, 10. err: 0.045 - std: 0.026 - time: 3.5
- 0, 11. err: 0.046 - std: 0.026 - time: 4.0
- 0, 12. err: 0.046 - std: 0.026 - time: 4.2
- 0, 13. err: 0.046 - std: 0.026 - time: 3.3
- 1, 0. err: 0.028 - std: 0.026 - time: 3.0
- 1, 1. err: 0.029 - std: 0.025 - time: 4.0
- 1, 2. err: 0.023 - std: 0.023 - time: 7.0
- 1, 3. err: 0.022 - std: 0.023 - time: 10.0
- 1, 4. err: 0.022 - std: 0.023 - time: 15.0
- 1, 5. err: 0.022 - std: 0.023 - time: 20.0
- 1, 6. err: 0.021 - std: 0.022 - time: 14.4
- 1, 7. err: 0.038 - std: 0.031 - time: 1.0
- 1, 8. err: 0.031 - std

,Exp,Mean Err,Median Err,Mean Std,time
0,False-15,0.045462,0.038978,0.027607,3.000472
1,False-20,0.044868,0.036347,0.027561,4.001764
2,False-35,0.043696,0.037682,0.027190,6.694528
3,False-50,0.043773,0.035797,0.027371,7.834844
4,False-75,0.043756,0.038307,0.027300,9.420223
5,False-100,0.043413,0.037580,0.027237,9.400645
6,False-200,0.043408,0.039817,0.027302,10.920258
7,True-15,0.045274,0.038455,0.028091,1.240987
8,True-20,0.044826,0.038855,0.027945,1.567120
9,True-35,0.044289,0.037076,0.027730,2.743674


#### Alternating Optimization

In [8]:
# # #####   Alternating Opt   #####
# Exps = [
#     {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'alt': False, 'epochs': 35}},
#     {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'alt': True, 'e_h': 1, 'e_W': 1, 'epochs': 35}},
#     {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'alt': True, 'e_h': 5, 'e_W': 5, 'epochs': 35}},
#     {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'alt': True, 'e_h': 10, 'e_W': 10, 'epochs': 35}},
#     {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'alt': True, 'e_h': 10, 'e_W': 10, 'epochs': 10}},
#     {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'alt': True, 'e_h': 5, 'e_W': 10, 'epochs': 15}},
#     {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'alt': True, 'e_h': 10, 'e_W': 5, 'epochs': 15}},

#     {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'alt': False, 'epochs': 35}},
#     {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'alt': True, 'e_h': 1, 'e_W': 1, 'epochs': 35}},
#     {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'alt': True, 'e_h': 5, 'e_W': 5, 'epochs': 35}},
#     {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'alt': True, 'e_h': 10, 'e_W': 10, 'epochs': 35}},
#     {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'alt': True, 'e_h': 10, 'e_W': 10, 'epochs': 10}},
#     {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'alt': True, 'e_h': 5, 'e_W': 10, 'epochs': 15}},
#     {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'alt': True, 'e_h': 10, 'e_W': 5, 'epochs': 15}},
# ]
# err, std, times = run_exp(data_p, default_arch_p, default_mod_p, Exps)
# leg = []
# for exp in Exps:
#     text = f'{exp["sel_GSOs"]}-{exp["mod_p"]["alt"]}-{exp["mod_p"]["epochs"]}'
#     if exp["mod_p"]["alt"]:
#         text += f'-{exp["mod_p"]["e_h"]}-{exp["mod_p"]["e_W"]}'
#     leg.append(text)

# display_data(leg, err, std, times)

- 0, 0. err: 0.043 - std: 0.031 - time: 27.8
- 0, 1. err: 0.049 - std: 0.030 - time: 52.1
- 0, 2. err: 0.027 - std: 0.029 - time: 246.1
- 0, 3. err: 0.033 - std: 0.028 - time: 482.9
- 0, 4. err: 0.030 - std: 0.028 - time: 138.3
- 0, 5. err: 0.030 - std: 0.029 - time: 156.3
- 0, 6. err: 0.041 - std: 0.030 - time: 156.2
- 0, 7. err: 0.032 - std: 0.031 - time: 9.8
- 0, 8. err: 0.111 - std: 0.050 - time: 18.0
- 0, 9. err: 0.037 - std: 0.028 - time: 87.7
- 0, 10. err: 0.029 - std: 0.030 - time: 175.7
- 0, 11. err: 0.096 - std: 0.044 - time: 47.7
- 0, 12. err: 0.068 - std: 0.036 - time: 56.6
- 0, 13. err: 0.034 - std: 0.028 - time: 54.3
- 1, 0. err: 0.044 - std: 0.026 - time: 28.1
- 1, 1. err: 0.046 - std: 0.027 - time: 52.3
- 1, 2. err: 0.043 - std: 0.026 - time: 230.4
- 1, 3. err: 0.044 - std: 0.026 - time: 485.1
- 1, 4. err: 0.043 - std: 0.025 - time: 138.5
- 1, 5. err: 0.044 - std: 0.025 - time: 155.9
- 1, 6. err: 0.051 - std: 0.027 - time: 156.3
- 1, 7. err: 0.042 - std: 0.026 - time: 9

,Exp,Mean Err,Median Err,Mean Std,time
0,False-False-35,0.050405,0.043974,0.027371,28.098897
1,False-True-35-1-1,0.059588,0.049267,0.028519,52.319649
2,False-True-35-5-5,0.046482,0.042572,0.026934,230.607375
3,False-True-35-10-10,0.046111,0.042861,0.026518,440.602201
4,False-True-10-10-10,0.055587,0.045063,0.028286,138.258532
5,False-True-15-5-10,0.051842,0.043760,0.028055,155.976822
6,False-True-15-10-5,0.052878,0.044066,0.026957,156.195269
7,True-False-35,0.049297,0.043897,0.028003,9.204408
8,True-True-35-1-1,0.059129,0.050531,0.029680,17.296460
9,True-True-35-5-5,0.047801,0.042742,0.027777,84.121649


#### Batch Size

In [12]:
#####   Batch Size   #####
Exps = [
    {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'bs': 5}},
    {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'bs': 10}},
    {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'bs': 25}},
    {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'bs': 50}},
    {'sel_GSOs': False, 'arc_p': {}, 'mod_p': {'bs': 100}},

    {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'bs': 5}},
    {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'bs': 10}},
    {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'bs': 25}},
    {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'bs': 50}},
    {'sel_GSOs': True, 'arc_p': {}, 'mod_p': {'bs': 100}},
]

err, std, times = run_exp(data_p, default_arch_p, default_mod_p, Exps)
leg = [f'{exp["sel_GSOs"]}-{exp["mod_p"]["bs"]}'  for exp in Exps]
display_data(leg, err, std, times)

- 0, 0. err: 0.048 - std: 0.028 - time: 15.9
- 0, 1. err: 0.048 - std: 0.028 - time: 11.6
- 0, 2. err: 0.047 - std: 0.029 - time: 10.0
- 0, 3. err: 0.048 - std: 0.030 - time: 9.8
- 0, 4. err: 0.048 - std: 0.029 - time: 10.0
- 0, 5. err: 0.050 - std: 0.031 - time: 17.7
- 0, 6. err: 0.049 - std: 0.031 - time: 9.6
- 0, 7. err: 0.049 - std: 0.031 - time: 4.3
- 0, 8. err: 0.049 - std: 0.032 - time: 2.7
- 0, 9. err: 0.050 - std: 0.032 - time: 2.7
- 1, 0. err: 0.040 - std: 0.030 - time: 6.9
- 1, 1. err: 0.033 - std: 0.022 - time: 5.3
- 1, 2. err: 0.033 - std: 0.023 - time: 5.4
- 1, 3. err: 0.034 - std: 0.024 - time: 4.1
- 1, 4. err: 0.033 - std: 0.023 - time: 10.0
- 1, 5. err: 0.032 - std: 0.023 - time: 9.5
- 1, 6. err: 0.032 - std: 0.024 - time: 4.3
- 1, 7. err: 0.032 - std: 0.024 - time: 3.9
- 1, 8. err: 0.032 - std: 0.022 - time: 2.7
- 1, 9. err: 0.032 - std: 0.022 - time: 2.7
- 2, 0. err: 0.010 - std: 0.013 - time: 20.0
- 2, 1. err: 0.011 - std: 0.014 - time: 11.0
- 2, 2. err: 0.013 - std

,Exp,Mean Err,Median Err,Mean Std,time
0,False-5,0.031121,0.029672,0.024849,13.910361
1,False-10,0.030199,0.028670,0.024566,10.027811
2,False-25,0.030603,0.029171,0.025367,9.168543
3,False-50,0.032295,0.032282,0.025938,8.629369
4,False-100,0.033377,0.031508,0.026302,9.890206
5,True-5,0.031612,0.030590,0.027144,17.209493
6,True-10,0.031675,0.029427,0.027723,8.595697
7,True-25,0.033318,0.032305,0.027780,4.190091
8,True-50,0.033013,0.032223,0.027961,2.684257
9,True-100,0.035718,0.036707,0.028532,2.624220


#### Number of Layers

In [13]:
#####   Number of Layers Size   #####
##### Total params (for L > 1): L*N + hid_dim**2*(L-2) + hid_dim
Exps = [
    {'sel_GSOs': False, 'arc_p': {'L': 1, 'hid_dim': 128}, 'mod_p': {}},
    {'sel_GSOs': False, 'arc_p': {'L': 2, 'hid_dim': 128}, 'mod_p': {}},
    {'sel_GSOs': False, 'arc_p': {'L': 3, 'hid_dim': 32}, 'mod_p': {}},
    {'sel_GSOs': False, 'arc_p': {'L': 4, 'hid_dim': 16}, 'mod_p': {}},

    {'sel_GSOs': True, 'arc_p': {'L': 1, 'hid_dim': 128}, 'mod_p': {}},
    {'sel_GSOs': True, 'arc_p': {'L': 2, 'hid_dim': 128}, 'mod_p': {}},
    {'sel_GSOs': True, 'arc_p': {'L': 3, 'hid_dim': 32}, 'mod_p': {}},
    {'sel_GSOs': True, 'arc_p': {'L': 4, 'hid_dim': 16}, 'mod_p': {}},
]

err, std, times = run_exp(data_p, default_arch_p, default_mod_p, Exps)
leg = [f'{exp["sel_GSOs"]}-{exp["arc_p"]["L"]}-{exp["arc_p"]["hid_dim"]}'  for exp in Exps]
display_data(leg, err, std, times)

- 0, 0. err: 0.041 - std: 0.031 - time: 2.6
- 0, 1. err: 0.028 - std: 0.029 - time: 15.3
- 0, 2. err: 0.028 - std: 0.031 - time: 14.1
- 0, 3. err: 0.029 - std: 0.033 - time: 17.1
- 0, 4. err: 0.046 - std: 0.034 - time: 2.1
- 0, 5. err: 0.029 - std: 0.031 - time: 4.5
- 0, 6. err: 0.029 - std: 0.031 - time: 5.2
- 0, 7. err: 0.030 - std: 0.032 - time: 5.7
- 1, 0. err: 0.064 - std: 0.022 - time: 2.5
- 1, 1. err: 0.056 - std: 0.019 - time: 14.8
- 1, 2. err: 0.057 - std: 0.019 - time: 14.0
- 1, 3. err: 0.057 - std: 0.019 - time: 16.9
- 1, 4. err: 0.066 - std: 0.023 - time: 2.3
- 1, 5. err: 0.056 - std: 0.019 - time: 4.2
- 1, 6. err: 0.058 - std: 0.019 - time: 3.4
- 1, 7. err: 0.057 - std: 0.019 - time: 5.8
- 2, 0. err: 0.052 - std: 0.023 - time: 2.7
- 2, 1. err: 0.046 - std: 0.020 - time: 11.0
- 2, 2. err: 0.046 - std: 0.020 - time: 13.9
- 2, 3. err: 0.046 - std: 0.020 - time: 16.9
- 2, 4. err: 0.050 - std: 0.023 - time: 2.6
- 2, 5. err: 0.047 - std: 0.022 - time: 2.4
- 2, 6. err: 0.048 - st

,Exp,Mean Err,Median Err,Mean Std,time
0,False-1-128,0.061054,0.050304,0.028651,2.603817
1,False-2-128,0.037108,0.028202,0.024542,14.082669
2,False-3-32,0.037275,0.027941,0.024197,12.695368
3,False-4-16,0.036816,0.029288,0.023770,16.351530
4,True-1-128,0.071526,0.059238,0.029262,2.287458
5,True-2-128,0.037786,0.029555,0.026024,3.931620
6,True-3-32,0.037220,0.029347,0.025671,4.908095
7,True-4-16,0.037712,0.029838,0.025646,6.641299


#### Size of Hidden Dimension

In [14]:
#####   Size of Hidden Dimension   #####
Exps = [
    {'sel_GSOs': False, 'arc_p': {'hid_dim': 32}, 'mod_p': {}},
    {'sel_GSOs': False, 'arc_p': {'hid_dim': 64}, 'mod_p': {}},
    {'sel_GSOs': False, 'arc_p': {'hid_dim': 128}, 'mod_p': {}},

    {'sel_GSOs': True, 'arc_p': {'hid_dim': 32}, 'mod_p': {}},
    {'sel_GSOs': True, 'arc_p': {'hid_dim': 64}, 'mod_p': {}},
    {'sel_GSOs': True, 'arc_p': {'hid_dim': 128}, 'mod_p': {}},
]

err, std, times = run_exp(data_p, default_arch_p, default_mod_p, Exps)
leg = [f'{exp["sel_GSOs"]}-{exp["arc_p"]["hid_dim"]}'  for exp in Exps]
display_data(leg, err, std, times)

- 0, 0. err: 0.064 - std: 0.020 - time: 6.7
- 0, 1. err: 0.064 - std: 0.022 - time: 10.0
- 0, 2. err: 0.064 - std: 0.021 - time: 13.4
- 0, 3. err: 0.064 - std: 0.021 - time: 2.4
- 0, 4. err: 0.065 - std: 0.022 - time: 1.9
- 0, 5. err: 0.065 - std: 0.023 - time: 1.6
- 1, 0. err: 0.023 - std: 0.024 - time: 7.6
- 1, 1. err: 0.025 - std: 0.025 - time: 10.0
- 1, 2. err: 0.023 - std: 0.024 - time: 15.3
- 1, 3. err: 0.027 - std: 0.026 - time: 3.5
- 1, 4. err: 0.029 - std: 0.026 - time: 3.1
- 1, 5. err: 0.030 - std: 0.026 - time: 1.8
- 2, 0. err: 0.080 - std: 0.023 - time: 3.2
- 2, 1. err: 0.079 - std: 0.023 - time: 5.2
- 2, 2. err: 0.079 - std: 0.023 - time: 9.5
- 2, 3. err: 0.079 - std: 0.024 - time: 2.0
- 2, 4. err: 0.079 - std: 0.024 - time: 1.6
- 2, 5. err: 0.080 - std: 0.023 - time: 2.9
- 3, 0. err: 0.057 - std: 0.013 - time: 4.1
- 3, 1. err: 0.056 - std: 0.014 - time: 7.6
- 3, 2. err: 0.057 - std: 0.013 - time: 5.8
- 3, 3. err: 0.056 - std: 0.013 - time: 2.4
- 3, 4. err: 0.055 - std: 0.

,Exp,Mean Err,Median Err,Mean Std,time
0,False-32,0.044473,0.024215,0.024252,6.821660
1,False-64,0.044575,0.025049,0.024471,8.886583
2,False-128,0.044489,0.026149,0.024028,13.680892
3,True-32,0.046277,0.031295,0.025739,3.387732
4,True-64,0.046174,0.028967,0.026084,3.437294
5,True-128,0.045984,0.030337,0.025687,3.579137


#### Bias

In [15]:
#####   Bias   #####
Exps = [
    {'sel_GSOs': False, 'arc_p': {'bias': False}, 'mod_p': {}},
    {'sel_GSOs': False, 'arc_p': {'bias': True}, 'mod_p': {}},
    {'sel_GSOs': True, 'arc_p': {'bias': False}, 'mod_p': {}},
    {'sel_GSOs': True, 'arc_p': {'bias': True}, 'mod_p': {}},
]

err, std, times = run_exp(data_p, default_arch_p, default_mod_p, Exps)
leg = [f'{exp["sel_GSOs"]}-{exp["arc_p"]["bias"]}'  for exp in Exps]
display_data(leg, err, std, times)

- 0, 0. err: 0.043 - std: 0.027 - time: 9.7
- 0, 1. err: 0.043 - std: 0.027 - time: 10.0
- 0, 2. err: 0.043 - std: 0.027 - time: 1.9
- 0, 3. err: 0.043 - std: 0.026 - time: 4.4
- 1, 0. err: 0.031 - std: 0.024 - time: 9.7
- 1, 1. err: 0.031 - std: 0.023 - time: 6.8
- 1, 2. err: 0.032 - std: 0.026 - time: 1.4
- 1, 3. err: 0.032 - std: 0.023 - time: 3.8
- 2, 0. err: 0.023 - std: 0.027 - time: 9.9
- 2, 1. err: 0.022 - std: 0.027 - time: 10.0
- 2, 2. err: 0.025 - std: 0.033 - time: 3.2
- 2, 3. err: 0.022 - std: 0.027 - time: 2.2
- 3, 0. err: 0.015 - std: 0.023 - time: 9.9
- 3, 1. err: 0.012 - std: 0.014 - time: 10.0
- 3, 2. err: 0.043 - std: 0.049 - time: 3.5
- 3, 3. err: 0.015 - std: 0.020 - time: 3.4
- 4, 0. err: 0.035 - std: 0.022 - time: 9.9
- 4, 1. err: 0.035 - std: 0.022 - time: 10.0
- 4, 2. err: 0.035 - std: 0.022 - time: 2.8
- 4, 3. err: 0.036 - std: 0.023 - time: 3.4
- 5, 0. err: 0.022 - std: 0.025 - time: 9.9
- 5, 1. err: 0.021 - std: 0.024 - time: 10.0
- 5, 2. err: 0.022 - std: 0

,Exp,Mean Err,Median Err,Mean Std,time
0,False-False,0.034434,0.030969,0.027029,9.092782
1,False-True,0.032876,0.031365,0.025026,9.250152
2,True-False,0.039272,0.042418,0.031707,2.441191
3,True-True,0.034852,0.034985,0.026941,3.570270
